In [82]:
import pandas as pd
import numpy as np
import schedule
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine
from scipy import stats
import function as fn


# 連接 sql 引擎
host='database-1.cyn7ldoposru.us-east-1.rds.amazonaws.com'
port='5432'
user='Yu'
password='m#WA12J#'
database="JQC_Revenue1"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
        user, password, host, port, database), echo=True)

# 讀取資料庫中的資料
db = pd.read_sql('tej_revenue', engine).drop('index', axis=1)
db['rev'] = round(db['rev']/1000, 2)

2023-03-27 09:45:22,810 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-27 09:45:22,812 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 09:45:23,214 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-27 09:45:23,215 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 09:45:23,623 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-27 09:45:23,625 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 09:45:24,033 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-27 09:45:24,034 INFO sqlalchemy.engine.Engine [generated in 0.00156s] {'name': 'tej_revenue'}
2023-03-27 09:45:24,648 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2023-03-27 09:45:24,650 INFO sqlalchemy.engine.Engine [generated i

2023-03-27 09:45:26,917 INFO sqlalchemy.engine.Engine [generated in 0.00148s] {'table_oid': 16512}
2023-03-27 09:45:27,118 INFO sqlalchemy.engine.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2023-03-27 09:45:27,119 INFO sqlalchemy.engine.Engine [generated in 0.00081s] {'table_oid': 16512}
2023-03-27 09:45:27,525 INFO sqlalchemy.engine.Engine SELECT tej_revenue.index, tej_revenue.st_code, tej_revenue.st_name, tej_revenue.rev_period, tej_revenue.declaration_date, tej_revenue.rev, tej_revenue.declaration_year, tej_revenue.declaration_month, tej_revenue.new_industry_name, tej_revenue.minor_industry_name 
FROM tej_revenue
2023-03-27 09:45:27,526 INFO sqlalchemy.engine.Engine [generated in 0.00112s] {}


In [ ]:
# 讀取更新報表
update = pd.read_excel("/Users/yuchun/Revenue/2022月營收更新報表_yuchun.xlsm", sheet_name='工作表1', header=6,
                       usecols=[3,4,5,6], names=['name', 'rev_period', 'declaration_date', 'rev'])
update[['st_code', 'st_name']] = update['name'].astype(str).str.split(n=1, expand=True)
update['declaration_date'] = pd.to_datetime(update['declaration_date'], format='%Y%m%d').dt.strftime('%Y/%m/%d')
update[['declaration_year', 'declaration_month', 'day']] = update['declaration_date'].astype(str).str.split('/', expand=True)
update = update.drop('day', axis=1)
update['rev_period'] = pd.to_datetime(update['rev_period'], format='%Y%m').dt.strftime('%Y/%m')
update = update[update.declaration_date != 'nan//'].drop('name', axis=1)


# 合併欄位及資料庫資訊
order = db.columns.values.tolist()
stock_info = db.loc[:, ['st_code', 'new_industry_name', 'minor_industry_name']].drop_duplicates(keep='first')
update = pd.merge(update, stock_info, on=['st_code'], how='inner')
update = update[order]
latest = update.rev_period.head(1).values[0]
update = update[~update.st_code.isin(db[db.rev_period == latest].st_code)]


#上傳新增的部分
if update.shape[0] == 0:
    predict_nxt_month = db
else:
    update.to_sql('tej_revenue', engine, if_exists='append')
    predict_nxt_month = pd.concat([db, update], axis=0)


# 預估次月營收
predict_nxt_month = predict_nxt_month.sort_values('rev_period')
predict_nxt_month = predict_nxt_month.groupby('st_code', as_index=False).apply(fn.get_st_mom_yoy).reset_index(drop=True)
predict_nxt_month = predict_nxt_month.groupby('st_code', as_index=False).apply(fn.get_predict).reset_index(drop=True)
predict_nxt_month = predict_nxt_month.drop_duplicates('st_code', keep='first')
predict_nxt_month = predict_nxt_month.drop(['declaration_date', 'declaration_year', 
                                            'declaration_month', 'Year', 'month'], axis=1)
predict_nxt_month = predict_nxt_month[predict_nxt_month.yoy <= 1000000000].sort_values('yoy', ascending=False)
predict_nxt_month.yoy = round(predict_nxt_month.yoy, 3)
predict_nxt_month.mom = round(predict_nxt_month.mom, 3)


# 輸出報表
filename = predict_nxt_month.rev_period.head(1).values[0][2:]
predict_nxt_month = predict_nxt_month.loc[:, ['st_code', 'st_name', 'rev_period', 'yoy', 'mom', 'rev', 'new_industry_name']]
predict_nxt_month = predict_nxt_month.rename({'st_code':'代號', 'st_name':'名稱', 'rev_period':'預估月', 
                                    'rev':'預估營收（百萬）', 'new_industry_name':'TEJ產業'}, axis=1)
predict_nxt_month.to_csv('/Users/yuchun/Revenue/Web2/models/{}營收預估.csv'.format(filename), 
                         encoding='utf_8_sig', 
                         header=True, 
                         index=False)


# 有發個股期的個股
futures_data = """
SELECT DISTINCT 標的證券 FROM 個股期歷史資料（近月及近二月）;
"""
st_future = pd.read_sql(futures_data, engine)
st_future['st_code'] = st_future['標的證券'].str[:4]
predict_nxt_month_f = predict_nxt_month[predict_nxt_month['代號'].isin(st_future.st_code)]
predict_nxt_month_f.to_csv('/Users/yuchun/Revenue/Web2/models/{}營收預估(個股期).csv'.format(filename), 
                           encoding='utf_8_sig', header=True, index=False)

In [22]:
# 最新公告營收報表
update = pd.read_excel("/Users/yuchun/Revenue/2022月營收更新報表_yuchun.xlsm", sheet_name='工作表1', header=6,
                       usecols=[3,4], names=['name', 'rev_period'])
update['rev_period'] = pd.to_datetime(update['rev_period'], format='%Y%m').dt.strftime('%Y/%m')


latest = update.sort_values('rev_period').rev_period.head(1).values[0]
latest_data = db.groupby('st_code', as_index=False).apply(fn.get_st_mom_yoy).reset_index(drop=True)
latest_data = latest_data.loc[:, ['st_code', 'st_name', 'rev_period', 'rev', 'mom', 'yoy', 'new_industry_name']]
latest_data = latest_data[(latest_data.rev_period == latest)]
latest_data.yoy = round(latest_data.yoy, 3)
latest_data.mom = round(latest_data.mom, 3)
latest_data = latest_data.rename({'st_code':'代號', 'st_name':'名稱', 'rev_period':'月份', 'rev':'預估營收（百萬）', 'new_industry_name':'TEJ產業別'}, axis=1)
latest_data.to_csv('/Users/yuchun/Revenue/Web2/models/{}最新營收公布.csv'.format(latest), 
                         encoding='utf_8_sig', 
                         header=True,
                         index=False)

,代號,名稱,月份,預估營收（百萬）,mom,yoy,new_industry_name
121,1101,台泥,2023/02,7306.07,-0.003,0.339,M1100 水泥工業
243,1102,亞泥,2023/02,6241.24,0.296,0.522,M1100 水泥工業
365,1103,嘉泥,2023/02,211.05,0.104,0.574,M1100 水泥工業
487,1104,環泥,2023/02,591.52,0.173,0.698,M1100 水泥工業
609,1108,幸福,2023/02,361.75,0.227,0.717,M1100 水泥工業
...,...,...,...,...,...,...,...
194579,9951,皇田,2023/02,411.04,0.115,0.216,M1500 電機機械
194701,9955,佳龍,2023/02,84.20,0.009,-0.350,M9900 其他
194822,9958,世紀鋼,2023/02,758.70,-0.299,-0.074,M2000 鋼鐵工業
194943,9960,邁達康,2023/02,61.63,0.049,0.314,M2900 貿易百貨


In [83]:
update = pd.read_excel("/Users/yuchun/Revenue/2022月營收更新報表_yuchun.xlsm", sheet_name='工作表1', header=6,
                       usecols=[3,4,5,6], names=['name', 'rev_period', 'declaration_date', 'rev'])

update[['st_code', 'st_name']] = update['name'].astype(str).str.split(n=1, expand=True)
update['declaration_date'] = pd.to_datetime(update['declaration_date'], format='%Y%m%d').dt.strftime('%Y/%m/%d')
update[['declaration_year', 'declaration_month', 'day']] = update['declaration_date'].astype(str).str.split('/', expand=True)
update = update.drop('day', axis=1)
update['rev_period'] = pd.to_datetime(update['rev_period'], format='%Y%m').dt.strftime('%Y/%m')
update = update[(update.declaration_date != 'nan//') & 
                (update.declaration_year != 'nan')].drop('name', axis=1)


# 合併欄位及資料庫資訊
order = db.columns.values.tolist()
stock_info = db.loc[:, ['st_code', 'new_industry_name', 'minor_industry_name']].drop_duplicates(keep='first')
update = pd.merge(update, stock_info, on=['st_code'], how='inner')
update = update[order]
latest = update.rev_period.head(1).values[0]
update = update[~update.st_code.isin(db[db.rev_period == latest].st_code)]
update

#上傳新增的部分
if update.shape[0] == 0:
    predict_nxt_month = db
else:
    update.to_sql('tej_revenue', engine, if_exists='append')
    predict_nxt_month = pd.concat([db, update], axis=0)


predict_nxt_month = predict_nxt_month.sort_values('rev_period')
predict_nxt_month = predict_nxt_month.groupby('st_code', as_index=False).apply(fn.get_st_mom_yoy).reset_index(drop=True)
predict_nxt_month = predict_nxt_month.groupby('st_code', as_index=False).apply(fn.get_predict).reset_index(drop=True)
predict_nxt_month = predict_nxt_month.drop_duplicates('st_code', keep='first')
predict_nxt_month = predict_nxt_month.drop(['declaration_date', 'declaration_year', 
                                            'declaration_month', 'Year', 'month'], axis=1)
predict_nxt_month = predict_nxt_month[predict_nxt_month.yoy <= 1000000000].sort_values('yoy', ascending=False)
predict_nxt_month.yoy = round(predict_nxt_month.yoy, 3)
predict_nxt_month.mom = round(predict_nxt_month.mom, 3)

predict_nxt_month

,st_code,st_name,rev_period,rev,new_industry_name,minor_industry_name,mom,yoy
876,1438,三地開發,* 2023/01,105.51,M2500 建材營造,M25A 建設,NaN,10550.000
15024,6186,新潤,* 2022/12,2443.08,M2500 建材營造,M25A 建設,168.188,847.292
852,1436,華友聯,* 2023/03,352.00,M2500 建材營造,M25A 建設,121.222,703.000
7188,3056,總太,* 2023/03,314.00,M2500 建材營造,M25A 建設,2.379,71.517
12672,5213,亞昕,* 2023/03,555.00,M2500 建材營造,M25A 建設,18.878,28.365
...,...,...,...,...,...,...,...,...
13716,5514,三豐,* 2023/02,0.00,M2500 建材營造,M25A 建設,-1.000,-1.000
10536,4157,太景*-KY,* 2023/03,0.00,M1722 生技醫療,M17C1 西藥,-1.000,-1.000
5328,2539,櫻花建,* 2022/12,0.00,M2500 建材營造,M25A 建設,-1.000,-1.000
14196,6020,大展證,* 2023/03,1.00,M2800 金融業,M30A 證券,-0.984,-1.060


In [154]:
predict_nxt_month[predict_nxt_month.rev_period == '* 2023/02']
# db[db.st_code == '1516'].sort_values('rev_period')
# db = db[db.rev_period < '2023/01']
fn.get_st_mom_yoy(db[db.st_code == '9105'].sort_values('rev_period'))
db

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name
0,2402,毅嘉,2013/01,2013/02/01,734.82,2013,2,M2328 電子零組件,M23B2C 軟板
1,3228,金麗科,2013/01,2013/02/01,24.04,2013,2,M2324 半導體,M23G5 IC設計
2,8255,朋程,2013/01,2013/02/02,281.13,2013,2,M1500 電機機械,M15D 汽車組件
3,1256,鮮活果汁-KY,2013/01,2013/02/04,151.27,2013,2,M1200 食品工業,M12C 飲料乳品
4,1560,中砂,2013/01,2013/02/04,333.67,2013,2,M1500 電機機械,M15A2 模具零件
...,...,...,...,...,...,...,...,...,...
1677,9951,皇田,2023/01,2023/02/10,368.74,2023,2,M1500 電機機械,M15D 汽車組件
1678,9955,佳龍,2023/01,2023/02/07,83.42,2023,2,M9900 其他,M99L1 貴金屬回收
1679,9958,世紀鋼,2023/01,2023/02/09,689.19,2023,2,M2000 鋼鐵工業,M20A2 鋼筋
1680,9960,邁達康,2023/01,2023/02/09,59.26,2023,2,M2900 貿易百貨,M29A9 其他零售


In [171]:
predict_nxt_month[(predict_nxt_month.rev_period == '2022/10')]

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,month,mom,yoy
117,1101,台泥,2022/10,2022/11/10,11368.10,2022,11,M1100 水泥工業,M11A 水泥製造,2022,10,0.0926,0.1885
239,1102,亞泥,2022/10,2022/11/10,7601.10,2022,11,M1100 水泥工業,M11A 水泥製造,2022,10,0.0868,-0.1960
361,1103,嘉泥,2022/10,2022/11/10,209.43,2022,11,M1100 水泥工業,M11A 水泥製造,2022,10,0.0203,0.0843
483,1104,環泥,2022/10,2022/11/10,634.98,2022,11,M1100 水泥工業,M11A 水泥製造,2022,10,0.0633,0.1360
605,1108,幸福,2022/10,2022/11/08,401.20,2022,11,M1100 水泥工業,M11A 水泥製造,2022,10,0.1574,0.4498
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195050,9951,皇田,2022/10,2022/11/10,432.84,2022,11,M1500 電機機械,M15D 汽車組件,2022,10,-0.0679,0.2458
195172,9955,佳龍,2022/10,2022/11/09,80.63,2022,11,M9900 其他,M99L1 貴金屬回收,2022,10,0.0183,-0.2943
195294,9958,世紀鋼,2022/10,2022/11/10,911.83,2022,11,M2000 鋼鐵工業,M20A2 鋼筋,2022,10,0.0096,0.0081
195416,9960,邁達康,2022/10,2022/11/10,79.00,2022,11,M2900 貿易百貨,M29A9 其他零售,2022,10,-0.3056,0.0102


In [155]:
predict_nxt_month = db

predict_nxt_month = predict_nxt_month.sort_values('rev_period')
predict_nxt_month = predict_nxt_month.groupby('st_code', as_index=False).apply(fn.get_st_mom_yoy).reset_index(drop=True)
# predict_nxt_month = predict_nxt_month.groupby('st_code', as_index=False).apply(fn.get_predict).reset_index(drop=True)
# predict_nxt_month = predict_nxt_month.drop_duplicates('st_code', keep='first')
# predict_nxt_month = predict_nxt_month.drop(['declaration_date', 'declaration_year', 
#                                             'declaration_month', 'Year', 'month'], axis=1)
# predict_nxt_month = predict_nxt_month[predict_nxt_month.yoy <= 1000000000].sort_values('yoy', ascending=False)
# predict_nxt_month.yoy = round(predict_nxt_month.yoy, 3)
# predict_nxt_month.mom = round(predict_nxt_month.mom, 3)

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,month,mom,yoy
0,1101,台泥,2013/01,2013/02/18,9134.46,2013,2,M1100 水泥工業,M11A 水泥製造,2013,01,NaN,NaN
1,1101,台泥,2013/02,2013/03/08,5540.35,2013,3,M1100 水泥工業,M11A 水泥製造,2013,02,-0.3935,NaN
2,1101,台泥,2013/03,2013/04/10,9457.97,2013,4,M1100 水泥工業,M11A 水泥製造,2013,03,0.7071,NaN
3,1101,台泥,2013/04,2013/05/10,9919.27,2013,5,M1100 水泥工業,M11A 水泥製造,2013,04,0.0488,NaN
4,1101,台泥,2013/05,2013/06/10,9543.78,2013,6,M1100 水泥工業,M11A 水泥製造,2013,05,-0.0379,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,9962,有益,2022/10,2022/11/04,307.43,2022,11,M2000 鋼鐵工業,M20A6 不鏽鋼,2022,10,0.1633,0.1299
195539,9962,有益,2022/11,2022/12/07,308.66,2022,12,M2000 鋼鐵工業,M20A6 不鏽鋼,2022,11,0.0040,-0.0841
195540,9962,有益,2022/12,2023/01/09,385.67,2023,1,M2000 鋼鐵工業,M20A6 不鏽鋼,2022,12,0.2495,0.5687
195541,9962,有益,2023/01,2023/02/07,281.17,2023,2,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,01,-0.2710,0.3300


In [178]:
# predict_data = predict_nxt_month.groupby('st_code', as_index=False).apply(get_predict).reset_index(drop=True)
predict_data[predict_data.st_code == '9105']
df = predict_nxt_month[predict_nxt_month.st_code == '9105']
data = df.sort_values('rev_period')
latest = data.rev_period.tail(1).values[0]
if data.shape[0] >= 10:
    a = data.groupby('Year', as_index=False).apply(get_ratio)
    ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
    predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
#     predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
#     predict['predict_month_rev'] = round(predict['predict_annul_rev'].tail(12).mean()* predict.wt_avg_ratio)
#     predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
#     predict['rev'] = predict['predict_month_rev']
#     predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
#     predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
#     predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
#     predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
#     predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
#     return predict.tail(12)
# else:
#     print(data.st_code.head(1).values[0], data.shape[0])
predict_nxt_month.groupby('st_code').agg({'st_code':'count'})#.groupby('count').agg({'count':'count'})

,st_code
st_code,
1101,122
1102,122
1103,122
1104,122
1108,122
...,...
9951,122
9955,122
9958,122


In [180]:
a = predict_nxt_month.groupby('st_code').agg({'st_code':'count'})
a[a.st_code == 121]

,st_code
st_code,
1591,121
3629,121
4142,121
4530,121
4712,121
4907,121
5205,121
6101,121
9105,121


In [186]:
db[db.st_code == '4712'].tail(12)

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name
174273,4712,南璋,2022/02,2022/03/08,0.04,2022,3,M1200 食品工業,M99ZA3 漁業
176241,4712,南璋,2022/03,2022/04/08,0.28,2022,4,M1200 食品工業,M99ZA3 漁業
177368,4712,南璋,2022/04,2022/05/06,0.18,2022,5,M1200 食品工業,M99ZA3 漁業
179280,4712,南璋,2022/05,2022/06/08,-0.05,2022,6,M1200 食品工業,M99ZA3 漁業
180884,4712,南璋,2022/06,2022/07/07,0.03,2022,7,M1200 食品工業,M99ZA3 漁業
182646,4712,南璋,2022/07,2022/08/08,0.03,2022,8,M1200 食品工業,M99ZA3 漁業
185368,4712,南璋,2022/08,2022/09/12,0.03,2022,9,M1200 食品工業,M99ZA3 漁業
187000,4712,南璋,2022/09,2022/10/11,0.03,2022,10,M1200 食品工業,M99ZA3 漁業
188087,4712,南璋,2022/10,2022/11/09,1.22,2022,11,M1200 食品工業,M99ZA3 漁業
190058,4712,南璋,2022/11,2022/12/09,0.32,2022,12,M1200 食品工業,M99ZA3 漁業


In [159]:
def get_yoy(data):
    data = data.sort_values('Year')
    data['yoy'] = round(data.rev.diff()/data.rev.shift(1), 4)
    return data

def get_ratio(data):
    ttl_rev = data.rev.sum()/len(data)*12
    data['ratio'] = round(data.rev/ttl_rev, 4)
    return data


def get_avg_ratio(data):
    if len(data) >= 5:
        weight = [1, 1, 1, 3, 5]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 4:
        weight = [1, 1, 2, 3]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 3:
        weight = [1, 1, 2]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 2:
        roll_num = len(data)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(data.ratio.mean(), 4)
        

def get_predict(data):
    data = data.sort_values('rev_period')
    if data.shape[0] >= 10:
        a = data.groupby('Year', as_index=False).apply(get_ratio)
        ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
        predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
        predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
        predict['predict_month_rev'] = round(predict['predict_annul_rev'].tail(12).mean()* predict.wt_avg_ratio)
        predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
        predict['rev'] = predict['predict_month_rev']
        predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
        predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
        predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
        predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
        predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
        return predict.tail(12)
    else:
        print(data.st_code.head(1).values[0], data.shape[0])

In [192]:
update = pd.read_excel('/Volumes/YUCHUN/20230327094322DataExport.xlsx')#.drop_duplicates(['年月','營收發布日','以合併為主單月營收(千元)'])
# update[update['代號'].astype(str).str[-1] == 'M']
update = update.loc[:, ['代號','名稱','年月','營收發布日','以合併為主單月營收(千元)']]
update = update.set_axis(['st_code', 'st_name', 'rev_period', 'declaration_date', 'rev'], axis=1)

# update[['declaration_year', 'declaration_month', 'day']] = update['declaration_date'].astype(str).str.split('/', expand=True)
# update['declaration_month'] = update.declaration_month.astype(int).astype(str)
# update = update.drop('day', axis=1)
# update = update[(update.declaration_date != 'nan//') & 
#                 (update.declaration_year != 'nan')]

# 合併欄位及資料庫資訊
# order = db.columns.values.tolist()
# stock_info = db.loc[:, ['st_code', 'new_industry_name', 'minor_industry_name']].drop_duplicates(keep='first')
# update = pd.merge(update, stock_info, on=['st_code'], how='inner')
# update = update[order]
# latest = update.rev_period.head(1).values[0]
# update = update[~update.st_code.isin(db[db.rev_period == latest].st_code)]
# update.rev = round(update.rev/1000, 2)
update[update.st_code == '4712']
# db = pd.concat([db, update], axis=0)
# update.to_sql('tej_revenue', engine, if_exists='append')

,st_code,st_name,rev_period,declaration_date,rev
1990,4712,南璋,2023/02,2023/03/10,0


In [194]:
predict_nxt_month

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,month,mom,yoy
0,1101,台泥,2013/01,2013/02/18,9134.46,2013,2,M1100 水泥工業,M11A 水泥製造,2013,01,NaN,NaN
1,1101,台泥,2013/02,2013/03/08,5540.35,2013,3,M1100 水泥工業,M11A 水泥製造,2013,02,-0.3935,NaN
2,1101,台泥,2013/03,2013/04/10,9457.97,2013,4,M1100 水泥工業,M11A 水泥製造,2013,03,0.7071,NaN
3,1101,台泥,2013/04,2013/05/10,9919.27,2013,5,M1100 水泥工業,M11A 水泥製造,2013,04,0.0488,NaN
4,1101,台泥,2013/05,2013/06/10,9543.78,2013,6,M1100 水泥工業,M11A 水泥製造,2013,05,-0.0379,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,9962,有益,2022/10,2022/11/04,307.43,2022,11,M2000 鋼鐵工業,M20A6 不鏽鋼,2022,10,0.1633,0.1299
195539,9962,有益,2022/11,2022/12/07,308.66,2022,12,M2000 鋼鐵工業,M20A6 不鏽鋼,2022,11,0.0040,-0.0841
195540,9962,有益,2022/12,2023/01/09,385.67,2023,1,M2000 鋼鐵工業,M20A6 不鏽鋼,2022,12,0.2495,0.5687
195541,9962,有益,2023/01,2023/02/07,281.17,2023,2,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,01,-0.2710,0.3300
